# Regularization (Ridge Regression) and Cross-validation



In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
sns.set_style("white")

### Load the data
We are going to use an advertisement dataset. The task is to figure out how different means of advertisement influence the amount of sales of a product.
    
Let's load the data with Pandas!

In [ ]:
# data loading
ad_df = pd.read_csv('Advertising.csv')
# view the first 5 rows 
ad_df.head()

### Fit a linear regression
Let's use TV and Radio advertisment as features to predict the sales.


In [ ]:
# making the data matrix and output
X = np.array(ad_df[["TV", "Radio"]])
y = np.array(ad_df["Sales"])
# check their dimensions
X.shape, y.shape

In [ ]:
# fit a linear regression model and check out the coefficients
LR = LinearRegression(fit_intercept=True)
LR.fit(X, y)
print("params: ", LR.coef_)
print("constant: ", LR.intercept_)
print("R^2 score: ", LR.score(X, y))

In [ ]:
# make some predictions
print("TV: ", 200, "Radio: ", 50, "Sales: ", LR.predict(np.array([200, 50]).reshape(-1,2)))
print("TV: ", 200, "Radio: ", 30, "Sales: ", LR.predict(np.array([200, 30]).reshape(-1,2)))

### Using polynomial features (polynomial combination of features)
Sometimes the data cannot be explained by just a linear model and using a linear model yields a high error. In these cases we can add polynomial features to the data which is equivalent to fitting polynomial functions on the data points.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# use polynomial features (nonlinear model)
poly = PolynomialFeatures(2)
X = np.array(ad_df[["TV", "Radio"]])
y = np.array(ad_df["Sales"])
X_poly = poly.fit_transform(X)

In [ ]:
X_poly.shape

In [ ]:
LR = LinearRegression(fit_intercept=False) # we don't need fit intercept sice polynomial features function adds a column of ones to the data 
LR.fit(X_poly, y)
print("params: ", LR.coef_)
print("R^2 score: ", LR.score(X_poly, y))

### Training/Test split and Overfitting

The standard practice in Machine Learning is to keep the test data out of the model training and try to evaluate the model's performance on the unseen data in the test set.

Now let's see the effect of adding more polynomial features. Adding too many of these features will result in complex models that may cause overfitting. We can check this by plotting curves of the training and the test errors.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# let's now use 1 feature from the data, namely TV, to predict the Sales
X = np.array(ad_df[["TV"]])
y = np.array(ad_df["Sales"])

In [ ]:
train_err = []
test_err = []
# fit models using polynomial features of orders between 1 and 6
# higher order polynomials have a higher model complexity 
for f in range(1,7):
    poly = PolynomialFeatures(f)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=1)
    LR = LinearRegression(fit_intercept=False)
    LR.fit(X_train, y_train)
    train_err.append(mean_squared_error(y_train, LR.predict(X_train)))
    test_err.append(mean_squared_error(y_test, LR.predict(X_test)))
# plot the training and test errors for the different models used    
plt.plot(range(1,7), train_err, label="train_error")
plt.plot(range(1,7), test_err, label="test_error")
plt.legend(fontsize=10)
plt.xlabel("polynomial degree")
plt.ylabel("error")

### Fit Ridge regression to avoid overfitting
To avoid overfitting we can penalize the norm of the weight vector to encourage simpler models. This is called regularization. We will use Ridge Regression (linear regression with L2 regularizer). By tuning the regularization parameter (the hyperparameter of the model that controls its complexity) $\alpha$ we can minimize the test error.

In [ ]:
from sklearn.linear_model import Ridge
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=1)
test_err = []
for alpha in np.linspace(0.1, 1, num=10):
    ridge_regression = Ridge(alpha=alpha)
    ridge_regression.fit(X_train, y_train)
    test_err.append(mean_squared_error(y_test, ridge_regression.predict(X_test)))
    

In [ ]:
# plot the test error for different values of alpha
plt.semilogx(np.logspace(-4, 2, num=10), test_err)
plt.xlabel("alpha")
plt.ylabel("test error");

### Use cross validation to chose the best value for the regularization parameter
One way to do a hyperparameter tuning is through cross validation. Let's do a 5-fold CV. We need to make sure that our model has a good generalisation power and performs well on unseen data. For this purpose we can use the RidgeCV function.

In [ ]:
# we use ridge regression with built in cross validation
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(fit_intercept=False, cv=5)
ridge.fit(X_train, y_train)

In [ ]:
# print he fitted regression coefficients
ridge.coef_

In [ ]:
# best regularizer
ridge.alpha_

In [ ]:
# fit the model with the best selected alpha
ridge_best = Ridge(alpha=ridge.alpha_)
ridge_best.fit(X_train, y_train)
# compute the MSE on the test set
mean_squared_error(y_test, ridge_best.predict(X_test))